In [19]:
import os
import pandas as pd
from string import digits
import re
from bs4 import BeautifulSoup

In [4]:
classes = os.listdir('./BBC News Summary/News Articles')
Articles_dir = './BBC News Summary/News Articles/'
Summaries_dir = './BBC News Summary/Summaries/'

articles = []
summaries = []
file_arr = []
for cls in classes:
    files = os.listdir(Articles_dir + cls)
    for file in files:
        article_file_path = Articles_dir + cls + '/' + file
        summary_file_path = Summaries_dir + cls + '/' + file
        try:
            with open (article_file_path,'r') as f:
                articles.append('.'.join([line.rstrip() for line in f.readlines()]))
            with open (summary_file_path,'r') as f:
                summaries.append('.'.join([line.rstrip() for line in f.readlines()]))
            file_arr.append(cls + '/' + file)
        except:
            pass
            
data = pd.DataFrame({'File_path':file_arr,'Articles': articles,'Summaries':summaries})
data.head()

,File_path,Articles,Summaries
0,business/001.txt,Ad sales boost Time Warner profit..Quarterly p...,TimeWarner said fourth quarter sales rose 2% t...
1,business/002.txt,Dollar gains on Greenspan speech..The dollar h...,The dollar has hit its highest level against t...
2,business/003.txt,Yukos unit buyer faces loan claim..The owners ...,Yukos' owner Menatep Group says it will ask Ro...
3,business/004.txt,High fuel prices hit BA's profits..British Air...,"Rod Eddington, BA's chief executive, said the ..."
4,business/005.txt,Pernod takeover talk lifts Domecq..Shares in U...,Pernod has reduced the debt it took on to fund...


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   File_path  2225 non-null   object
 1   Articles   2225 non-null   object
 2   Summaries  2225 non-null   object
dtypes: object(3)
memory usage: 52.3+ KB


In [7]:
data.describe()

,File_path,Articles,Summaries
count,2225,2225,2225
unique,2225,2127,2081
top,business/001.txt,Ray DVD beats box office takings..Oscar-nomina...,Although the two partially-paralysed people pe...
freq,1,2,2


In [8]:
# data.to_csv('news.csv')

In [9]:
data.shape

(2225, 3)

In [10]:
contractions_dictionary = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "this's": "this is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "here's": "here is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}


In [11]:
def Filter(text):
    text=text.lower()
    text=' '.join([contractions_dictionary[i] if i in contractions_dictionary.keys() else i for i in text.split()])
    text=re.sub(r'\(.*\)',"",text)
    text=re.sub("'s","",text)
    text=re.sub('"','',text)
    text=' '.join([i for i in text.split() if i.isalpha()])
    text=re.sub('[^a-zA-Z]'," ",text)
    
    return text

data['File_path'] = data['File_path'].apply(Filter)
data['Articles'] = data['Articles'].apply(Filter)
data['Summaries'] = data['Summaries'].apply(Filter)

In [17]:
# data = data.drop(['File_path'], axis=1)
data

,Articles,Summaries
0,ad sales boost time warner profits at us media...,timewarner said fourth quarter sales rose to f...
1,dollar gains on greenspan dollar has hit its h...,the dollar has hit its highest level against t...
2,yukos unit buyer faces loan owners of embattle...,owner menatep group says it will ask rosneft t...
3,high fuel prices hit ba airways has blamed hig...,rod ba chief said the results were respectable...
4,pernod takeover talk lifts in uk drinks and fo...,pernod has reduced the debt it took on to fund...
...,...,...
2220,bt program to beat dialler is introducing two ...,bt is introducing two initiatives to help beat...
2221,spam tempt net users across the world continue...,a third of them read unsolicited junk and buy ...
2222,be careful how you new european directive coul...,this goes to the heart of the european and eve...
2223,us cyber security chief man making sure us com...,amit yoran was director of the national cyber ...


In [21]:
# Remove HTML tags from the data frame if they are present

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
data['Summary'] = data['Summaries'].apply(strip_html)
data['Article'] = data['Articles'].apply(strip_html)

data=data.drop(['Summaries', 'Articles'],axis=1)

In [22]:
data

,Summary,Article
0,timewarner said fourth quarter sales rose to f...,ad sales boost time warner profits at us media...
1,the dollar has hit its highest level against t...,dollar gains on greenspan dollar has hit its h...
2,owner menatep group says it will ask rosneft t...,yukos unit buyer faces loan owners of embattle...
3,rod ba chief said the results were respectable...,high fuel prices hit ba airways has blamed hig...
4,pernod has reduced the debt it took on to fund...,pernod takeover talk lifts in uk drinks and fo...
...,...,...
2220,bt is introducing two initiatives to help beat...,bt program to beat dialler is introducing two ...
2221,a third of them read unsolicited junk and buy ...,spam tempt net users across the world continue...
2222,this goes to the heart of the european and eve...,be careful how you new european directive coul...
2223,amit yoran was director of the national cyber ...,us cyber security chief man making sure us com...
